In [ ]:
#velocity



import rasterio
import numpy as np
from pathlib import Path

# ==============================================================================
# 1. Configuration (CHANGE THESE VALUES)
# ==============================================================================

# **REQUIRED CHANGE:** Replace this with the exact, full path to your unwrapped
# displacement TIFF file. This file contains the total displacement (in meters).
# NOTE: Using a raw string (r"...") for robust Windows path handling.
INPUT_DISPLACEMENT_TIF = Path(r"D:\blatten_project\data\blatten_phase1\sentinel1\slc\final\slc_20250510-20250522_Orb_Stack_ifg_deb_flt_unwrapped_dsp_TC\S1A_IW_SLC__1SDV_20250510T172342_20250510T172409_059135_07564A_4C6F_Orb_Stack_ifg_deb_flt_dsp_TC.tif")

# Output velocity path (as defined in your original script)
# NOTE: Using a raw string (r"...") for robust Windows path handling.
OUTPUT_VELOCITY_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\slc_outputs\New folder\velocity_mm_per_day.tif")

# Temporal Baseline (Time difference between 2025-05-02 and 2025-05-26)
TEMPORAL_BASELINE_DAYS = 12

# Ensure the output directory structure exists
OUTPUT_VELOCITY_TIF.parent.mkdir(parents=True, exist_ok=True)


# ==============================================================================
# 2. Velocity Calculation Function
# ==============================================================================

def calculate_velocity(input_path: Path, output_path: Path, days: int):
    """
    Calculates the daily Line-of-Sight (LOS) velocity in mm/day from 
    a total displacement raster (assumed to be in meters).

    Args:
        input_path (Path): Path to the input displacement raster (in meters).
        output_path (Path): Path to save the output velocity raster (in mm/day).
        days (int): The temporal baseline (time difference) between the acquisitions.
    """
    print(f"--- Starting Velocity Calculation ---")
    print(f"Input Displacement File: {input_path}")
    print(f"Temporal Baseline: {days} days")
    
    # Check if the input file exists before proceeding
    if not input_path.exists():
        print(f"\nERROR: Input file not found at {input_path}")
        print("Please ensure INPUT_DISPLACEMENT_TIF points to a valid file.")
        return

    try:
        # Open the input raster and read its metadata
        with rasterio.open(input_path) as src:
            # Read the data into a NumPy array
            displacement_m = src.read(1).astype(np.float64)
            profile = src.profile
            nodata_value = profile['nodata']

        print(f"Raster Shape: {displacement_m.shape}")
        print(f"NoData Value (Input): {nodata_value}")
        
        # --- The Core Calculation ---
        # Formula: Velocity (mm/day) = (Displacement (m) / Time (days)) * 1000
        
        # 1. Convert total displacement (m) to daily displacement (m/day)
        daily_displacement_m = displacement_m / days
        
        # 2. Convert daily displacement (m/day) to velocity (mm/day)
        velocity_mm_per_day = daily_displacement_m * 1000
        
        print("Calculation complete.")

        # --- Update Raster Profile for Output ---
        # Update the profile to reflect the new data type and NoData value.
        # Since velocity can be negative (movement away/towards sensor), we use 
        # Float32, which is standard for scientific rasters.
        profile.update(dtype=rasterio.float32, 
                       count=1, 
                       nodata=-9999.0) # Use a common NoData value for float output
        
        # Handle the NoData pixels
        if nodata_value is not None:
             # Set NoData pixels in the output array to the new NoData value
             velocity_mm_per_day[displacement_m == nodata_value] = profile['nodata']

        # Write the new velocity raster
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(velocity_mm_per_day.astype(rasterio.float32), 1)

        print(f"\nSUCCESS: Velocity map created at: {output_path}")
        print(f"Units: mm/day")

    except rasterio.RasterioIOError as e:
        print(f"\nERROR: Could not read or write raster file. Details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


# ==============================================================================
# 3. Execution Block
# ==============================================================================

# Ensure the INPUT_DISPLACEMENT_TIF path is correctly configured before running
calculate_velocity(
    INPUT_DISPLACEMENT_TIF, 
    OUTPUT_VELOCITY_TIF, 
    TEMPORAL_BASELINE_DAYS
)


In [ ]:
#coherence extract


import rasterio
from pathlib import Path

# ==============================================================================
# 1. Configuration (CHANGE THESE VALUES)
# ==============================================================================

# **REQUIRED CHANGE:** Replace this with the exact, full path to your multi-band TIFF.
# This file contains both the phase/displacement and the coherence bands.
INPUT_MULTI_BAND_TIF = Path(r"D:\blatten_project\data\blatten_phase1\sentinel1\slc\final\slc_20250510-20250522_Orb_Stack_ifg_deb_flt_unwrapped_dsp_TC\S1A_IW_SLC__1SDV_20250510T172342_20250510T172409_059135_07564A_4C6F_Orb_Stack_ifg_deb_flt_TC.tif")

# Output path for the single-band Coherence TIFF
OUTPUT_COH_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\slc_outputs\New folder\coherence_band.tif")

# Coherence Band Index: Coherence is usually the second band in geocoded InSAR outputs (index 2).
COHERENCE_BAND_INDEX = 4

# Ensure the output directory structure exists
OUTPUT_COH_TIF.parent.mkdir(parents=True, exist_ok=True)


# ==============================================================================
# 2. Coherence Export Function
# ==============================================================================

def export_single_band(input_path: Path, output_path: Path, band_index: int):
    """
    Extracts a specified band from a multi-band raster and saves it as a new
    single-band GeoTIFF.
    """
    print(f"--- Starting Coherence Band Export ---")
    print(f"Input File: {input_path}")
    
    if not input_path.exists():
        print(f"\nERROR: Input file not found at {input_path}")
        print("Please ensure INPUT_MULTI_BAND_TIF points to a valid file.")
        return

    try:
        # Open the multi-band input raster
        with rasterio.open(input_path) as src:
            print(f"Total Bands Detected: {src.count}")
            
            if band_index > src.count:
                print(f"\nERROR: Band index {band_index} exceeds the total number of bands ({src.count}).")
                print("Coherence is typically Band 2. Please check your file structure.")
                return

            # Read the target band (Coherence)
            coherence_data = src.read(band_index)
            profile = src.profile

        print(f"Successfully read Band {band_index} (Coherence).")

        # --- Update Raster Profile for Output ---
        # Crucially, set the band count to 1 for the new single-band file
        profile.update(count=1, 
                       dtype=coherence_data.dtype)
        
        # Write the new single-band coherence raster
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(coherence_data, 1)

        print(f"\nSUCCESS: Single-band coherence map exported to: {output_path}")

    except rasterio.RasterioIOError as e:
        print(f"\nERROR: Could not read or write raster file. Details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


# ==============================================================================
# 3. Execution Block
# ==============================================================================

export_single_band(
    INPUT_MULTI_BAND_TIF, 
    OUTPUT_COH_TIF, 
    COHERENCE_BAND_INDEX
)


In [ ]:
import rasterio
import numpy as np
from pathlib import Path
from rasterio.enums import Resampling
# Importing the dedicated function for aligning rasters
from rasterio.warp import reproject 

# ==============================================================================
# 1. Configuration (CHANGE THESE VALUES)
# ==============================================================================

# **Master Image (MST):** The earlier date (2025-04-28).
# CORRECTED PATH to use 'grd_outputs' instead of 'slc_outputs'
INPUT_MASTER_AMP_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\grd_outputs\Subset_S1A_IW_GRDH_1SDV_20250428T172343_20250428T172408_058960_074FA0_65DF_Orb_NR_Cal_Spk_TC_dB.tif")

# **Slave Image (SLV):** The later date (2025-05-22).
# CORRECTED PATH to use 'grd_outputs' instead of 'slc_outputs'
INPUT_SLAVE_AMP_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\grd_outputs\Subset_S1A_IW_GRDH_1SDV_20250522T172342_20250522T172407_059310_075C59_45E7_Orb_NR_Cal_Spk_TC_dB.tif")

# Output path for the single-band Amplitude Difference TIFF (Difference = SLV - MST)
OUTPUT_DIFF_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\slc_outputs\New folder\amplitude_difference_dB.tif")

# Ensure the output directory structure exists
OUTPUT_DIFF_TIF.parent.mkdir(parents=True, exist_ok=True)


# ==============================================================================
# 2. Amplitude Difference Calculation Function
# ==============================================================================

def calculate_amplitude_difference(master_path: Path, slave_path: Path, output_path: Path):
    """
    Calculates the amplitude difference (Slave - Master) between two amplitude 
    rasters. It ensures spatial alignment by explicitly using rasterio.warp.reproject 
    to align the slave to the master's dimensions and transform before subtraction.
    """
    print(f"--- Starting Amplitude Difference Calculation (SLV - MST) ---")
    print(f"Master File: {master_path.name}")
    print(f"Slave File: {slave_path.name}")
    
    if not master_path.exists() or not slave_path.exists():
        print("\nERROR: One or both input amplitude files were not found.")
        return

    try:
        # 1. Open and Read Master Raster (The spatial template)
        with rasterio.open(master_path) as master_src:
            master_amp = master_src.read(1).astype(np.float32)
            profile = master_src.profile
            master_nodata = profile.get('nodata')
            
            # Extract essential spatial information from the Master
            master_shape = master_amp.shape
            master_transform = master_src.transform
            master_crs = master_src.crs
            print(f"Master Shape: {master_shape}")


        # 2. Open and Read Slave Raster (Aligned to the Master)
        with rasterio.open(slave_path) as slave_src:
            # Initialize an empty array for the resampled slave data, 
            # matching the Master's shape and datatype.
            slave_amp = np.empty(master_shape, dtype=np.float32)

            # Use rasterio.warp.reproject to align the slave data to the master's profile
            reproject(
                source=rasterio.band(slave_src, 1), # Source data (Band 1 of Slave)
                destination=slave_amp,            # Target array (matching master dimensions)
                src_transform=slave_src.transform,
                src_crs=slave_src.crs,
                dst_transform=master_transform,   # Target transform (from Master)
                dst_crs=master_crs,               # Target CRS (from Master)
                resampling=Resampling.nearest,
                num_threads=4                     # Use 4 threads for better performance
            )

        print(f"Slave Resampled Shape: {slave_amp.shape}")
        
        # 3. Handle NoData Values
        # Combine NoData masks from both input files.
        
        # Check if master has a defined nodata value
        if master_nodata is not None:
            mask = (master_amp == master_nodata) | (slave_amp == master_nodata)
        else:
            # If no nodata is defined, rely on standard numpy masking (though less common for GeoTIFFs)
            mask = np.zeros_like(master_amp, dtype=bool)

        # 4. Perform Subtraction: SLAVE - MASTER
        amplitude_diff = slave_amp - master_amp
        
        print("Subtraction complete.")

        # 5. Update Raster Profile for Output
        # The output difference is float, and we use a new nodata value.
        output_nodata = -9999.0
        
        # Update profile to match master's dimensions, transform, CRS, but set count=1 
        # and new dtype/nodata for the difference product.
        profile.update(dtype=rasterio.float32, 
                       count=1, 
                       nodata=output_nodata,
                       transform=master_transform,
                       crs=master_crs,
                       width=master_shape[1],
                       height=master_shape[0])
        
        # Apply the combined NoData mask to the output array
        amplitude_diff[mask] = output_nodata

        # 6. Write the new Amplitude Difference raster
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(amplitude_diff, 1)

        print(f"\nSUCCESS: Amplitude difference map created at: {output_path}")
        print(f"Formula: Slave ({slave_path.name}) - Master ({master_path.name})")

    except Exception as e:
        print(f"\nAn error occurred during raster processing: {e}")


# ==============================================================================
# 3. Execution Block
# ==============================================================================

calculate_amplitude_difference(
    INPUT_MASTER_AMP_TIF, 
    INPUT_SLAVE_AMP_TIF, 
    OUTPUT_DIFF_TIF
)


In [ ]:
import rasterio
import numpy as np
from pathlib import Path
from rasterio.enums import Resampling
# Importing the dedicated function for aligning rasters
from rasterio.warp import reproject 

# ==============================================================================
# 1. Configuration (CHANGE THESE VALUES)
# ==============================================================================

# TARGET BAND INDEX: Band index to use for the change calculation (1-based).
# Band 4 (Red) is selected for general land change detection.
# If your bands are 1, 2, 3, 4, 12:
# 1: B1 (Aerosol) | 2: B2 (Blue) | 3: B3 (Green) | 4: B4 (Red) | 5: B12 (SWIR)
TARGET_BAND_INDEX = 4 # Using Band 4 (Red)

# **Master Image (MST):** The earlier date (2025-04-30).
INPUT_MASTER_AMP_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\s2_outputs\Subset_S2A_MSIL2A_20250430T102701_N0511_R108_T32TMS_20250430T190517_resampled.tif")

# **Slave Image (SLV):** The later date (2025-05-18).
INPUT_SLAVE_AMP_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\s2_outputs\Subset_S2C_MSIL2A_20250518T102621_N0511_R108_T32TMS_20250518T143414_resampled.tif")

# Output path for the single-band Spectral Difference TIFF (Difference = SLV - MST)
# This will be your S2_CHANGE_TIF input.
OUTPUT_DIFF_TIF = Path(r"D:\blatten_project\data\blatten_phase1\derived\slc_outputs\New folder\s2_amplitude_diff.tif")

# Ensure the output directory structure exists
OUTPUT_DIFF_TIF.parent.mkdir(parents=True, exist_ok=True)


# ==============================================================================
# 2. Raster Difference Calculation Function
# ==============================================================================

def calculate_spectral_difference(master_path: Path, slave_path: Path, output_path: Path):
    """
    Calculates the spectral difference (Slave - Master) between two Sentinel-2 
    rasters for the specified TARGET_BAND_INDEX. It ensures spatial alignment 
    by explicitly using rasterio.warp.reproject.
    """
    band_index = TARGET_BAND_INDEX
    print(f"--- Starting Spectral Difference Calculation (Band {band_index}, SLV - MST) ---")
    print(f"Master File: {master_path.name}")
    print(f"Slave File: {slave_path.name}")
    
    if not master_path.exists() or not slave_path.exists():
        print("\nERROR: One or both input raster files were not found.")
        return

    try:
        # 1. Open and Read Master Raster (The spatial template)
        with rasterio.open(master_path) as master_src:
            # Read the specified band
            master_amp = master_src.read(band_index).astype(np.float32)
            profile = master_src.profile
            master_nodata = profile.get('nodata')
            
            # Extract essential spatial information from the Master
            master_shape = master_amp.shape
            master_transform = master_src.transform
            master_crs = master_src.crs
            print(f"Master Shape: {master_shape}")


        # 2. Open and Read Slave Raster (Aligned to the Master)
        with rasterio.open(slave_path) as slave_src:
            # Initialize an empty array for the resampled slave data, 
            # matching the Master's shape and datatype.
            slave_amp = np.empty(master_shape, dtype=np.float32)

            # Use rasterio.warp.reproject to align the slave data to the master's profile
            reproject(
                source=rasterio.band(slave_src, band_index), # Source data (Target Band of Slave)
                destination=slave_amp,                      # Target array (matching master dimensions)
                src_transform=slave_src.transform,
                src_crs=slave_src.crs,
                dst_transform=master_transform,             # Target transform (from Master)
                dst_crs=master_crs,                         # Target CRS (from Master)
                resampling=Resampling.nearest,
                num_threads=4                               # Use 4 threads for better performance
            )

        print(f"Slave Resampled Shape: {slave_amp.shape}")
        
        # 3. Handle NoData Values
        # Combine NoData masks from both input files.
        if master_nodata is not None:
            mask = (master_amp == master_nodata) | (slave_amp == master_nodata)
        else:
            mask = np.zeros_like(master_amp, dtype=bool)

        # 4. Perform Subtraction: SLAVE - MASTER
        spectral_diff = slave_amp - master_amp
        
        print("Subtraction complete.")

        # 5. Update Raster Profile for Output
        output_nodata = -9999.0
        
        profile.update(dtype=rasterio.float32, 
                       count=1, 
                       nodata=output_nodata,
                       transform=master_transform,
                       crs=master_crs,
                       width=master_shape[1],
                       height=master_shape[0])
        
        # Apply the combined NoData mask to the output array
        spectral_diff[mask] = output_nodata

        # 6. Write the new Spectral Difference raster
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(spectral_diff, 1)

        print(f"\nSUCCESS: Spectral difference map (Band {band_index}) created at: {output_path}")
        print(f"Formula: Slave ({slave_path.name}) - Master ({master_path.name})")

    except Exception as e:
        print(f"\nAn error occurred during raster processing: {e}")


# ==============================================================================
# 3. Execution Block
# ==============================================================================

calculate_spectral_difference(
    INPUT_MASTER_AMP_TIF, 
    INPUT_SLAVE_AMP_TIF, 
    OUTPUT_DIFF_TIF
)


In [ ]:
# Cell A: imports & parameters (CHANGE THESE)
import os, json
from pathlib import Path
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.features import shapes
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import fiona
from fiona.crs import from_epsg
import scipy.ndimage as ndi

# ------------------ CHANGE THESE PATHS ------------------
# Input rasters (aligned: same transform, CRS, shape)
VEL_TIF = Path(r"derived/velocity/velocity_mm_per_day.tif")        # velocity (mm/day)
COH_TIF = Path(r"derived/slc_outputs/mean_coherence.tif")         # mean coherence (0..1)
AMP_DIFF_TIF = Path(r"derived/grd_outputs/amplitude_diff.tif")    # amplitude diff (dB) post - pre
# OPTIONAL: sentinel-2 change mask (0/1) or amplitude diff from S2
S2_CHANGE_TIF = Path(r"derived/s2_outputs/s2_amplitude_diff.tif") # optional (if you have)
# Output GeoJSON path (where frontend will fetch it)
OUT_GEOJSON = Path(r"frontend/derived/risk/high_risk.geojson")
# Temporary working folder
WORK_DIR = Path("derived/risk")
WORK_DIR.mkdir(parents=True, exist_ok=True)
# ------------------------------------------------------

# ------------------ THRESHOLDS (tune these) ------------------
VEL_THRESHOLD_MM = 10.0       # mm/day threshold for high velocity
COH_THRESHOLD = 0.35         # coherence below this considered degraded
AMP_DB_THRESHOLD = 2.0       # amplitude dB change threshold
MIN_AREA_M2 = 1000           # minimum polygon area to keep (square meters)
# final mask combination weights (for confidence scoring)
W_VEL, W_COH, W_AMP = 0.45, 0.35, 0.20
# ------------------------------------------------------------

print("Inputs:\n", VEL_TIF, COH_TIF, AMP_DIFF_TIF)
print("Output GeoJSON:", OUT_GEOJSON)


In [ ]:
/* ---------------------- S2 Animation + Change Overlay ----------------------
   Drop this into your index.js after your map is created (map variable available).
   IMPORTANT: Edit the section labeled "CHANGE THESE VALUES" below.
   ------------------------------------------------------------------------ */

(function() {
  // --------------------- CHANGE THESE VALUES ---------------------
  // Directory where your PNG frames live (relative to site root)
    const FRAME_PATH = 'https://postimg.cc/gallery/PhHtqFY';

  // Ordered list of PNG filenames (chronological)
  const frames = [
    'https://i.postimg.cc/qvw6DqZd/Subset-S2-C-MSIL2-A-20250408-T103051-N0511-R108-T32-TMS-20250408-T161415-resampled-B12-B2-B1-rgb.png',
    'https://i.postimg.cc/8CyKG1sg/Subset-S2-A-MSIL2-A-20250430-T102701-N0511-R108-T32-TMS-20250430-T190517-resampled-B12-B2-B1-rgb.png',
    'https://i.postimg.cc/bJWmG3QG/Subset-S2-C-MSIL2-A-20250518-T102621-N0511-R108-T32-TMS-20250518-T143414-resampled-B12-B2-B1-rgb.png',
    'https://i.postimg.cc/4dqWKBz7/Subset-S2-A-MSIL2-A-20250530-T103041-N0511-R108-T32-TMS-20250530-T142711-resampled-B12-B2-B1-rgb.png'  // example extra you added
  ];

  // Corresponding human-readable dates for UI (strings)
  const dates = [
    '2025-04-08',
    '2025-04-30',
    '2025-05-18',
    '2025-05-30'
  ];

  // The index (0-based) of the first frame that shows the detected change.
  // Option A: set this manually (fast hack). Set to the index where change appears.
  // Option B: if your GeoJSON features contain a 'date' property that matches the dates above,
  //           the code will prefer that and compute this automatically.
  let changeFrameIndex = 1; // <-- EDIT: set to the index of the change frame (e.g. 1 for 2025-05-18)

  // Pixel overlay bounds [southWest, northEast] = [[south, west], [north, east]]
  // REPLACE these four numbers with the bounds you computed earlier.
  const OVERLAY_BOUNDS = [[46.32, 7.706], [46.48, 8.07]]; // <-- EDIT: [south, west], [north, east]

  // Path to change GeoJSON (precomputed polygons)
  const CHANGE_GEOJSON_PATH = '/derived/risk/high_risk.geojson'; // relative to site root
  // ---------------------------------------------------------------------

  // UI state
  let currentIndex = 0;
  let timer = null;
  let intervalMs = 1000; // default speed: 1000ms per frame
  const minInterval = 200;
  const maxInterval = 4000;

  // Create UI controls container (inject into dashboard-right or map)
  const controlsContainer = document.createElement('div');
  controlsContainer.id = 's2-anim-controls';
  controlsContainer.style.padding = '10px';
  controlsContainer.style.background = 'rgba(255,255,255,0.96)';
  controlsContainer.style.borderRadius = '10px';
  controlsContainer.style.boxShadow = '0 6px 18px rgba(0,0,0,0.12)';
  controlsContainer.style.marginBottom = '12px';
  // append to right sidebar if available, otherwise to body
  const rightSidebar = document.querySelector('.dashboard-right') || document.body;
  rightSidebar.insertBefore(controlsContainer, rightSidebar.firstChild);

  // Build inner HTML for controls
  controlsContainer.innerHTML = `
    <div style="font-weight:700;margin-bottom:6px">S2 Time-lapse</div>
    <div style="display:flex;gap:8px;align-items:center;margin-bottom:6px">
      <button id="anim-prev" class="btn-ghost">◀</button>
      <button id="anim-play" class="btn-ghost">Play ▶</button>
      <button id="anim-pause" class="btn-ghost">Pause ⏸</button>
      <button id="anim-next" class="btn-ghost">▶</button>
    </div>
    <div style="display:flex;align-items:center;gap:8px;margin-bottom:6px">
      <label style="font-size:12px;">Speed</label>
      <input id="anim-speed" type="range" min="${minInterval}" max="${maxInterval}" value="${intervalMs}" step="100" style="flex:1" />
    </div>
    <div style="margin-bottom:6px">
      <div style="font-size:13px">Frame: <span id="anim-frame-label">-</span></div>
      <div style="font-size:12px;color:#666">Date: <span id="anim-date-label">-</span></div>
    </div>
    <div id="anim-alert" style="display:none;background:#ffdddd;color:#700; padding:8px;border-radius:6px;font-weight:700;">
      ⚠️ Change detected in AOI
    </div>
  `;

  // create the image overlay (initial frame will be frames[0])
  const firstFrameUrl = FRAME_PATH + frames[0];
  let frameOverlay = L.imageOverlay(firstFrameUrl, OVERLAY_BOUNDS, {opacity: 0.95}).addTo(map);

  // load and add the change GeoJSON but keep invisible initially
  let changeLayer = null;
  fetch(CHANGE_GEOJSON_PATH).then(r => {
    if (!r.ok) throw new Error('No geojson');
    return r.json();
  }).then(geojson => {
    // style for hidden and highlighted
    const baseStyle = {color: '#ff3b3b', weight: 2, fillOpacity: 0.0};
    const highlightStyle = {color: '#ff3b3b', weight: 3, fillOpacity: 0.18};
    changeLayer = L.geoJSON(geojson, {
      style: baseStyle,
      onEachFeature: function(feature, layer) {
        // store optional feature date/frameIndex for auto detection
        // allow properties.date in ISO 'YYYY-MM-DD' or properties.frameIndex (0-based)
        layer.featureDate = feature.properties && feature.properties.date ? feature.properties.date : null;
        layer.featureFrameIndex = (feature.properties && typeof feature.properties.frameIndex !== 'undefined') ? feature.properties.frameIndex : null;
      }
    }).addTo(map);
    // initially hide by setting opacity zero (we don't remove it so map bounds remain)
    changeLayer.eachLayer(l => {
      l.setStyle({fillOpacity:0, opacity:0});
    });

    // If any feature provides a 'date' property that matches our dates list, compute changeFrameIndex automatically
    // (prefer automatic detection over manual)
    const featureDates = [];
    changeLayer.eachLayer(l => { if (l.featureDate) featureDates.push(l.featureDate); });
    if (featureDates.length > 0) {
      // find earliest matching date among features that appears in our frames' dates list
      for (let i=0;i<dates.length;i++) {
        if (featureDates.includes(dates[i])) { changeFrameIndex = i; break; }
      }
      console.log('Auto-detected changeFrameIndex =', changeFrameIndex);
    }
  }).catch(err => {
    console.log('No change geojson loaded:', err.message);
  });

  // UI helper refs
  const elFrame = document.getElementById('anim-frame-label');
  const elDate = document.getElementById('anim-date-label');
  const elPlay = document.getElementById('anim-play');
  const elPause = document.getElementById('anim-pause');
  const elPrev = document.getElementById('anim-prev');
  const elNext = document.getElementById('anim-next');
  const elSpeed = document.getElementById('anim-speed');
  const elAlert = document.getElementById('anim-alert');

  // update overlay to a given index
  function showFrame(idx) {
    if (idx < 0) idx = 0;
    if (idx >= frames.length) idx = frames.length - 1;
    currentIndex = idx;
    const url = FRAME_PATH + frames[idx];
    // replace overlay image src quickly by removing old overlay and adding new
    map.removeLayer(frameOverlay);
    frameOverlay = L.imageOverlay(url, OVERLAY_BOUNDS, {opacity:0.95}).addTo(map);

    // update labels
    elFrame.textContent = `${idx+1} / ${frames.length}`;
    elDate.textContent = dates[idx] || 'unknown';

    // show/hide change polygon & alert
    const isChangeFrame = (idx >= changeFrameIndex);
    if (changeLayer) {
      changeLayer.eachLayer(layer => {
        // If feature has frameIndex or date we can do per-feature decisions
        let show = false;
        if (layer.featureFrameIndex !== null) {
          show = (idx >= layer.featureFrameIndex);
        } else if (layer.featureDate !== null) {
          // compare dates
          const fi = dates.indexOf(layer.featureDate);
          show = (fi !== -1 && idx >= fi);
        } else {
          // fallback: if idx >= changeFrameIndex then show all
          show = (idx >= changeFrameIndex);
        }
        if (show) {
          layer.setStyle({opacity:1, fillOpacity:0.12});
        } else {
          layer.setStyle({opacity:0, fillOpacity:0});
        }
      });
    }

    // Toggle alert box
    if (isChangeFrame) elAlert.style.display = 'block';
    else elAlert.style.display = 'none';
  }

  // playback functions
  function play() {
    if (timer) return;
    timer = setInterval(() => {
      const nextIndex = (currentIndex + 1) < frames.length ? currentIndex + 1 : currentIndex;
      showFrame(nextIndex);
      if (nextIndex >= frames.length - 1) {
        pause(); // stop at last frame
      }
    }, intervalMs);
  }
  function pause() {
    if (timer) clearInterval(timer);
    timer = null;
  }
  function prev() {
    pause();
    showFrame(Math.max(0, currentIndex - 1));
  }
  function next() {
    pause();
    showFrame(Math.min(frames.length - 1, currentIndex + 1));
  }

  // bind UI
  elPlay.addEventListener('click', () => { play(); });
  elPause.addEventListener('click', () => { pause(); });
  elPrev.addEventListener('click', () => { prev(); });
  elNext.addEventListener('click', () => { next(); });
  elSpeed.addEventListener('input', (ev) => {
    intervalMs = parseInt(ev.target.value, 10);
    if (timer) { pause(); play(); } // restart timer with new speed
  });

  // show first frame now
  showFrame(0);

  // expose for debugging
  window.s2anim = { showFrame, play, pause, next, prev, frames, dates };

})();
